In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re

In [6]:
def file_to_sentence_list(file_path):
	with open(file_path, 'r') as file:
		text = file.read()

	sentences = [sentence.strip() for sentence in re.split(
		r'(?<=[.!?])\s+', text) if sentence.strip()]

	return sentences

In [7]:
file_path = 'pizza.txt'

In [8]:
text_data = file_to_sentence_list(file_path)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1


In [9]:
input_sequences = []
for line in text_data:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
	input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

In [10]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)


model = Sequential()
model.add(Embedding(total_words, 10,
					input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',
			optimizer='adam', metrics=['accuracy'])


In [12]:
model.fit(X, y, epochs=10, verbose=1)

Epoch 1/10
51/51 [==============================] - 1s 24ms/step - loss: 4.0821 - accuracy: 0.1505
Epoch 2/10
51/51 [==============================] - 1s 23ms/step - loss: 3.9785 - accuracy: 0.1634
Epoch 3/10
51/51 [==============================] - 1s 22ms/step - loss: 3.8775 - accuracy: 0.1701
Epoch 4/10
51/51 [==============================] - 1s 22ms/step - loss: 3.7780 - accuracy: 0.1800
Epoch 5/10
51/51 [==============================] - 1s 22ms/step - loss: 3.6825 - accuracy: 0.1972
Epoch 6/10
51/51 [==============================] - 1s 21ms/step - loss: 3.5935 - accuracy: 0.2076
Epoch 7/10
51/51 [==============================] - 1s 21ms/step - loss: 3.5004 - accuracy: 0.2156
Epoch 8/10
51/51 [==============================] - 1s 21ms/step - loss: 3.4084 - accuracy: 0.2340
Epoch 9/10
51/51 [==============================] - 1s 21ms/step - loss: 3.3185 - accuracy: 0.2506
Epoch 10/10
51/51 [==============================] - 1s 21ms/step - loss: 3.2285 - accuracy: 0.2801


In [13]:
seed_text = "Pizza have different "
next_words = 5


In [14]:
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences(
		[token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted_probs = model.predict(token_list)
	predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
	seed_text += " " + predicted_word

1/1 [==============================] - 0s 33ms/step


In [15]:
print("Next predicted words:", seed_text)

Next predicted words: Pizza have different  become a symbol of pizza
